## Pupil Segmentation FullyRot - LUT

> Author: [Zhengyi Zhan](mailto:zhanzy@zju.edu.cn)
>
> Date: Sept 15, 2024

### Introduction

This script uses a binary grating on an SLM to perform pupil segmentation for local and non-linear LUT correction.

It implements a 'FullyRot' pattern where one mask is moved in a rotational order.

Key Features:
- Utilizes HPC_SLM_Calib for SLM control and GxCam for camera operations
- Generates rotational coordinates for grating mask placement
- Applies LUT changes to the moving mask
- Captures and saves image data for further analysis

### Initialize Camera & SLM

In [1]:
from hpc_slm.hpc_slm import *
from gxipy.MyGxCam import GxCam

monitor_idx = 2             # Index of the monitor connected to SLM (via DVI)
size = [1024, 1272]         # SLM resolution [height, width]

cam_model_query = '1070'    # Camera model identifier

# Initialize SLM & grating object
slm = HPC_SLM_Calib(monitor_idx, 0, size)
slm.display_img(np.zeros(size, dtype=np.uint8)) # Display a blank image on SLM to verify correct monitor

# Initialize cam
cam = GxCam(cam_model_query)
cam.set_trigger_mode(True)      # Enable trigger mode for synchronized capture

### Set Grating Cooridnates Parameters

In [2]:
center_x = 636
center_y = 460

beam_diameter = 310     # Diameter of the beam
grating_size = 29       # Size of each grating mask
d1 = 36                 # Distance from the center to the center of the first circle

grating_type = 'binary' # The type of the grating
grating_step = 2        # Step size for the grating

turns = 5               # Number of rotational turns
# numbers = 9
numbers = np.array([9, 18, 26, 30, 40]) # Custom numbers of points for each turn

beam_diameter += grating_size   # Adjust beam diameter to include grating size

from hpc_slm.gratings import *
import matplotlib.pyplot as plt

g = Gratings(size)

# Generate rotational coordinates and preview image
cx, cy, img = g.gen_rotational_coors(grating_size, d1, beam_diameter, 
                                     center_x, center_y, turns, numbers, img_flag=True)

### Preview Rotational Cooridnates

In [ ]:
plt.figure(figsize=(9, 6))
plt.imshow(img)
plt.show()

### Acquisition

In [ ]:
from tqdm import tqdm
import time
from pathlib import Path

# start, end, numbers
lut = np.linspace(0, 255, 15, endpoint = True)

slm_delay = 0.1         # Delay between SLM updates (in seconds)
frame_per_postion = 1   # Number of frames to capture per position

# phase_step = 4          # Number of phase steps (use 1 to disable phase shift)

# Load SLM calibration file
# calib_file = Path(r'D:\.MINFLUX\CalibData\slm-calib\yzt-642-100ms.h5')  # SLM calibration file
# flat_flag = True        # Apply flat field correction
# local_flag = True       # Use local LUT for calibration
# global_lut = 200        # Global LUT value (only used when local_flag is False)
# slm.load_calib_file(calib_file)

# Generate a whole grating pattern
grating = g.gen_grating(grating_type, beam_diameter,
                        grating_step = grating_step,
                        circle_mask = True, phase_inverse = False)

# Open camera stream
cam.open_stream()
time.sleep(0.1) # Short delay to ensure camera is ready

# Place the binary grating into the specified cooridnates of canvas
whole_grating_canvas = g.place_patterns(cx[0:1], cy[0:1], grating)

# Get one frame from camera to initialize memory and set up image array
tmp = cam.get_snapshot()
imgs = np.zeros(tmp.shape + (len(lut), len(cx),), dtype = tmp.dtype)

time.sleep(slm_delay)

# Main acquisition loop
for i in tqdm(range(len(cx))):
    # Generate canvas mask for one circle and apply it
    grating_mask = g.gen_canvas_mask(np.array([grating_size]), cx[i:i+1], cy[i:i+1], circle_mask=True)
    grating_canvas = g.apply_canvas_mask(whole_grating_canvas, grating_mask)
 
    for j, value in enumerate(lut):
        # Send pattern to SLM
        slm.phs_to_display(grating_canvas, False, False, value)

        time.sleep(slm_delay)   # Wait for SLM to ensure the pattern is changed

        # Capture one frame from camera
        imgs[:, :, j, i] = cam.get_snapshot()

# Close camera stream
cam.close_stream()

### Data Preview

In [ ]:
# Preview the first 9 images
plt.figure(figsize=(9, 9))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(imgs[:, :, 2, i+1])
plt.show()

### Save Data

In [10]:
from scipy.io import savemat
from datetime import datetime

# Generate filename with current timestamp
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
file_name = Path(f"pupilseg_lut_{current_time}.mat")
final_path = Path(r"D:\BeamSense\PyPupilSegLUT") / file_name

# Prepare the structs of metadata
grating_para = {
    'type': grating_type,
    'size': grating_size,
    'step': grating_step,
    'beam_diameter': beam_diameter,
    'd1': d1,
    'turns': turns,
    'numbers': numbers
}

slm_para = {
    # 'local': local_flag,
    # 'flat': flat_flag,
    # 'calib_file': str(calib_file),
    'delay': slm_delay
}

# Save data and metadata to '.mat'
savemat(str(final_path), {'imgs': imgs,
                          'cx': cx, 'cy': cy, 
                          'lut': lut,
                          'grating_para': grating_para, 
                          'slm_para': slm_para})

### Release Camera & SLM

In [5]:
del cam
del slm